## Data-driven optimization and decision making - Assignment 3
Juha Reinikainen

Use EI and the mean prediction to solve any single objective benchmark problem (e.g.
Ackley, Rosenblock, sphere etc.) with any single objective optimizer (preferably GA). Set
max exact function evaluations to 50 (start with 50 design points). Was the solutions
found by EI better? (you can implement EI is you wish to)

In [5]:
import numpy as np
from pymoo.factory import get_problem
from scipy.stats import qmc
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor

import problems


In [6]:
def online_datadriven_rosenbrock_optimization(use_EI = False):
    """
    find minimum of rosenbrock
    use_EI use expected improvement strategy to choose point to evaluate
    otherwise mean approximation is used
    """
    seed = 1
    max_eval = 50
    # rosenbrock = get_problem("ackley", n_var=1, a=20, b=1/5, c=2 * np.pi)
    #x = 1, f = 0
    rosenbrock = get_problem("rosenbrock", n_var=2)
    surrogate = GaussianProcessRegressor()
    
    optimizer = GA(pop_size=100)
    optimizer_acquisition = GA(pop_size=100)

    #generate initial sample
    lhs = qmc.LatinHypercube(rosenbrock.n_var, seed=seed)
    X = lhs.random(50)
    X = qmc.scale(X, rosenbrock.xl, rosenbrock.xu)
    y = rosenbrock.evaluate(X)

    #Build surrogates with initial data
    surrogate.fit(X, y)

    history = {
        "X": [],
        "y": []
    }

    surrogateProblem = problems.Surrogate(rosenbrock, surrogate)
    for _ in range(max_eval):
        #find optimum of surrogate
        best = minimize(surrogateProblem, optimizer, ("n_gen", 10), seed=seed)
        history["X"].append(best.X)
        history["y"].append(best.F)

        #choose which point to evaluate
        if use_EI:
            #construct EI rosenbrock with found optimum
            ei = problems.ExpectedImprovement(rosenbrock, surrogate, best.F)

            #optimize considering expected improvement
            res = minimize(ei, optimizer_acquisition, ("n_gen", 10), seed=seed)
            x_t = res.X
        else:
            x_t = best.X

        #print(res.X, -res.F, end = ", ")
        #evaluate the chosen point with real function
        y_t = rosenbrock.evaluate(x_t)

        #add newly evaluated to data
        X = np.append(X, [x_t], axis=0)
        y = np.append(y, [y_t], axis=0)
        #Rebuild the surrogates with the new data
        surrogate.fit(X, y)

    # do final optimization
    best = minimize(surrogateProblem, optimizer, ("n_gen", 10), seed=seed)
    history["X"].append(best.X)
    history["y"].append(best.F)

    return best.X, best.F, history

In [7]:
x1,y1, hist1 = online_datadriven_rosenbrock_optimization(False)
x2,y2, hist2 = online_datadriven_rosenbrock_optimization(True)

In [8]:
x_true = np.ones(x1.shape)
y_true = np.zeros(y1.shape)
print("mean", x1,y1, np.linalg.norm(x_true - x1))
print("EI  ", x2,y2, np.linalg.norm(x_true - x2))

mean [0.97923458 0.95661389] [0.00099707] 0.04809944780227716
EI   [1.05692369 1.1119617 ] [0.03909372] 0.12560146341718723


Mean prediction is a little better than expected improvement on 2 dimensional Rosenbrock function. 